In [1]:
import turicreate as tc
import numpy as np

In [2]:
sales = tc.SFrame('home_data/')
sales['floors'] = sales['floors'].astype(float).astype(int)

In [3]:
def get_numpy_data(data_frame, features, output):
    data_frame['constant'] = 1
    features = ['constant'] + features
    features_sframe = data_frame[features]
    features_matrix = features_sframe.to_numpy()
    output_sarray = data_frame[output]
    output_array = output_sarray.to_numpy()
    print(features_matrix)
    return features_matrix, output_array

In [4]:
def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return predictions

In [8]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix / norms
    return normalized_features, norms

In [9]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print(features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print(norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


In [10]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
simple_feature_matrix, output = get_numpy_data(sales, simple_features, my_output)

[[1.00e+00 1.18e+03 3.00e+00]
 [1.00e+00 2.57e+03 3.00e+00]
 [1.00e+00 7.70e+02 2.00e+00]
 ...
 [1.00e+00 1.02e+03 2.00e+00]
 [1.00e+00 1.60e+03 3.00e+00]
 [1.00e+00 1.02e+03 2.00e+00]]


In [11]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [12]:
weights = np.array([1., 4., 1.])

In [15]:
prediction = predict_output(simple_feature_matrix, weights)
prediction

array([0.02675867, 0.04339256, 0.01990703, ..., 0.02289873, 0.03178473,
       0.02289873])

In [26]:
ro = [0 for i in range(simple_feature_matrix.shape[1])]
for i in range((simple_feature_matrix.shape)[1]):
    ro[i] = sum(simple_feature_matrix[:, i]*(output- prediction + weights[i]*simple_feature_matrix[:, i]))
ro

[79400300.03492953, 87939470.7729908, 80966698.67596525]

In [28]:
print(2*ro[1])
print(2*ro[2])

175878941.5459816
161933397.3519305


In [60]:
def to_zero(value, l1_penalty):
    return value >= -l1_penalty/2 and value <= l1_penalty/2

In [74]:
l1_penalties = np.logspace(1, 15, num=6)
for l1 in l1_penalties:
    print(to_zero(ro[1], l1), to_zero(ro[2], l1))

False False
False False
False False
True True
True True
True True


In [94]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = sum(feature_matrix[:, i]* (output - prediction + weights[i]*feature_matrix[:, i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = (ro_i + l1_penalty/2)
    elif ro_i > l1_penalty/2.:
        new_weight_i = (ro_i - l1_penalty/2)
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [95]:
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                    np.array([1., 1.]), np.array([1., 4.]), 0.1))
# should print 0.425558846691

0.4255588466910251


In [ ]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    